In [ ]:
# Detectie de trasaturi faciale (dlib)
# index 1 - coloana
# index 2 - linie
# matrice 68x2

import numpy as np
import dlib
import cv2
import matplotlib.pyplot as plt
import pdb
 
def rect_to_bb(rect):
	# ia un dreptunghi prezis de dlib si il converteste intr-un format
    # acceptat de opencv (x,y,w,h)
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
#	pdb.set_trace()
	# returneaza un tuplu (x, y, w, h)
	return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
	# initializeaza o lista pt cele 68 de coordonate
	coords = np.zeros((68, 2), dtype=dtype)
 
	# parcurge toate cele 68 de landmarkuri
    # shape este un obiect dlib, part este metoda care acceseaza fiecare punct
    # shape.part(i) acceseaza fiecare pereche(i) din care acceseaza atat coord x cat si y
	for i in range(0, 68):
		coords[i] = [shape.part(i).x, shape.part(i).y]
 
	# returneaza coordonatele sub forma unei matrici 68 x 2
    # prima coloana reprezinta coloana iar cea de-a doua linia
	return coords

    
# initializeaza detectorul de fete din dlib si predictoul punctelor faciale
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/drive/MyDrive/Facultate/IVOM/Lab_4/shape_predictor_68_face_landmarks.dat')

# incarca imaginea si o transforma in grayscale
image = cv2.imread('/content/drive/MyDrive/Facultate/IVOM/Lab_4/face1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


# foloseste detectorul initializat anterior sa gaseasca toate fetele din imagine
rects = detector(gray, 1)

# parcurge toate fetele din imagine(i pozitie,rect fata propriu-zisa)
for (i, rect) in enumerate(rects):
	# determina coordonatele faciale pentru fiecare fata detectata
	# foloseste functia shape_to_np sa converteasca perechile de coord (x,y) in numpy array
	shape = predictor(gray, rect)
	shape = shape_to_np(shape)
 
	# converteste dreptunghiul detectata intr-un format convenabil opencv
	(x, y, w, h) = rect_to_bb(rect)
	cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
 
	# arata numarul fetei detectate
	cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
 
	# deseneaza punctele faciale pentru fiecare fata
	for (x, y) in shape:
		cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
 
# afiseaza imgainea cu fetele detectate si punctele faciale aferente
from google.colab.patches import cv2_imshow
cv2_imshow(cv2.resize(image,(600,600)))
# cv2.namedWindow("Output", cv2.WINDOW_NORMAL)
# cv2.waitKey(1000)  

In [1]:
#Pt conectarea driveului la COLAB
#fisierele voastre se afla la /content/gdrive/My Drive/

#from google.colab import drive
#
#drive.mount('/content/gdrive')


import pdb
import numpy as np
import dlib
import cv2
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'dlib'

In [ ]:
def rect_to_bb(rect):
	# ia un dreptunghi prezis de dlib si il converteste intr-un format
    # acceptat de opencv (x,y,w,h)
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
 
	# returneaza un tuplu (x, y, w, h)
	return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
	# initializeaza o lista pt cele 68 de coordonate
	coords = np.zeros((68, 2), dtype=dtype)
 
	# parcurge toate cele 68 de landmarkuri
    # shape este un obiect dlib, part este metoda care acceseaza fiecare punct
    # shape.part(i) acceseaza fiecare pereche din care acceseaza atat coord x cat si y
	for i in range(0, 68):
		coords[i] = [shape.part(i).x, shape.part(i).y]
 
	# returneaza coordonatele sub forma unei matrici 68 x 2
    # prima coloana reprezinta coloana iar cea de-a doua linia
	return coords

In [ ]:
def decupeaza_ochi_gura(gray_image, color_image, facial_image):
    
    #PARAMETERS : imaginea gray,imaginea color, coordonatele componentei faciale(ochi_st,ochi_dr,gura)
    #RETURN : portiunea din imaginea gray asociata fiecarei trasaturi faciale
    L,C,_ = np.shape(color_image)

    #ca sa desenez dreptunghiul pt ochi si gura am nevoie de un colt st-sus si dr-jos
    #ma folosesc de vectorul de coordonate asociat fiecarei componente faciale pt a determina
    #punctele relevante pentru dreptunghiul de incadrare
    # NU UITATI ca pe coloana 1 avem coordonatele coloanelor, iar pe coloana 2 coord liniilor
    col_min = min(facial_image[:,0] - round(0.02*C))
    col_max = max(facial_image[:,0] + round(0.02*C))
    lin_min = min(facial_image[:,1] - round(0.02*L))
    lin_max = max(facial_image[:,1] + round(0.02*L))

    # deseneaza dreptunghi, pozitia colturilor este de tipul (coloana,linie) si nu (linie,coloana)
    # primul param este imaginea color pt ca pe ea vreau sa marchez ochii si gura
    # cele 2 tupluri sunt asociate pt coltul din st_sus si dr_jos
    #foloseste culoarea rosie(BGR), iar linia are grosime 3
    cv2.rectangle(color_image, (col_min,lin_min), (col_max,lin_max), (0,0,255), 3)
    
    #returneaza decupajul ochilor si al gurii folosind imaginea grayscale
    return gray_image[lin_min:lin_max,col_min:col_max], color_image[lin_min:lin_max,col_min:col_max]


In [ ]:
def detecteaza_irisi(gray_image, color_image, coord_ochi, por_decupata):
    
    
    #PARAMETERS: img_gray,img_color,coordonate pt fiecare ochi,portiunea decupata pt fiec trasatura din img initiala
    L,C,_ = np.shape(color_image)
    
    # ca sa scalez la intreaga imagine am nevoie de coltul din stanga sus al dreptunghiului
    # care imi contine toate punctele faciale asociate trasaturii curente
    min_linii     = np.min(coord_ochi[:,1]) - round(L*0.02)
    min_coloane   = np.min(coord_ochi[:,0]) - round(C*0.02)
    
    #proiectiile integrale pe H si V cu np.sum() pe img gray
    suma_coloane  = np.sum(por_decupata, axis = 0)
    suma_linii    = np.sum(por_decupata, axis = 1)  
    
    #
    # dim= 
    # capat=int(np.floor(dim/2))
    # suma_coloane_new=np.zeros()
    # suma_linii_new=np.zeros()
    
    #filtrare temporala folosin np.mean() pt fereastra respectiva
    # for i in range(capat,len(suma_coloane)-capat):
        
    # for i in range(capat,len(suma_linii)-capat)
    # NU mai e nevoie de foruri 
  
    # calculeaza pozitia punctului minim din dreptinghi(prob iris)
    #folosim np.where care primeste ca parametru o conditie
    #rezultatul va fi pozitia pe care se afla valoarea minima
    linie_iris    = np.where(suma_linii==np.min(suma_linii))
    coloana_iris  = np.where(suma_coloane==np.min(suma_coloane))
    print(coloana_iris)

    #decide unde se uita persoana in functie de proiectia pe V
    if coloana_iris[0][0] > len(suma_coloane)//2:
      print("Stanga")
    else:
      print("Dreapta")
    
    #deseneaza cu albastru irisul in imaginea color  
    #in tuplu treb specificata poz irisului calculata
    #NU UITATI ca trebuie sa scalati la dim imaginii originale, deci la pozitia determinata trebuie adunata
    #pozitia coltului din st-sus pt vectorul de coordonate
    cv2.circle(color_image,(min_coloane + coloana_iris[0][0], min_linii + por_decupata.shape[0]//2),3, (255,0,0),2)
    
    #plotez proiectiile pe H si pe V
    plt.figure()
    plt.title('Proiectie orizontala')
    plt.scatter(range(0,len(suma_coloane)), suma_coloane)
    plt.figure()
    plt.title('Proiectie verticala')
    plt.scatter(range(0,len(suma_linii)), suma_linii)

In [ ]:
# initializeaza detectorul de fete din dlib si predictoul punctelor faciale
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/drive/MyDrive/Facultate/IVOM/Lab_4/shape_predictor_68_face_landmarks.dat')

# incarca imaginea si o transforma in grayscale
#image = cv2.imread('C:\\Users\\Andrei\\Desktop\\IVOM_LAB\\labIVOMS\\Images\\BioID_F1_1.jpg')
image = cv2.imread('/content/drive/MyDrive/Facultate/IVOM/Lab_4/face2.png')
#image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


# foloseste detectorul initializat anterior sa gaseasca toate fetele din imagine
rects = detector(gray, 1)

# parcurge toate fetele din imagine(i pozitie,rect fata propriu-zisa)
for (i, rect) in enumerate(rects):
	# determina coordonatele faciale pentru fiecare fata detectata
	# foloseste functia shape_to_np sa converteasca perechile de coord (x,y) in numpy array
	shape = predictor(gray, rect)
	shape = shape_to_np(shape)
    
    #Decupez din shape portiunile aferente ochilor si gurii
    #Trebuie sa va uitati la ordinea punctelor din platforma si sa selectati landmarkurile
    #corespunzatoare pt fiecare componenta faciala
	ochi_st = shape[36:42,:]
	ochi_dr = shape[42:48,:]
	gura    = shape[48:68,:]
    
    #foloseste fctia de detectie pentru componentele faciale folosind parametrii corespunzatori
	os=decupeaza_ochi_gura(gray, image, ochi_st)[0]
	od=decupeaza_ochi_gura(gray, image, ochi_dr)[0]
	g=decupeaza_ochi_gura(gray, image, gura)[0]
    
    #apeleaza fctia de detectie de iris cu param coresp
    # trebuie sa folositi si marimile calcutate mai sus (os si od)
	detecteaza_irisi(gray, image, ochi_st, os)
	detecteaza_irisi(gray, image, ochi_dr, od)
 
	# converteste dreptunghiul detectata intr-un format convenabil opencv
	(x, y, w, h) = rect_to_bb(rect)
	cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
 
	# arata numarul fetei detectate
	cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
 
	# deseneaza punctele faciale pentru fiecare fata
	for (x, y) in shape:
		cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
 
# afiseaza imgainea cu fetele detectate si punctele faciale aferente

#pt COLAB    
cv2_imshow(cv2.resize(image,(600,600)))
        
# cv2.imshow("Output", cv2.resize(image,(600,600)))
# cv2.namedWindow("Output", cv2.WINDOW_NORMAL)
# cv2.waitKey(1000)  
#cv2.destroyAllWindows()